Cours : SQL pour Débutant(e)s Jour 3 - Approfondissement des Requêtes SQL
Lien : https://www.youtube.com/watch?v=4Ro69oqZ20s

Github le coin stat : https://github.com/LeCoinStat/LeCoinStat/tree/main/Bien_Debuter_SQL
Fichiers Github récupérés : https://github.com/LeCoinStat/LeCoinStat/tree/main/Bien_Debuter_SQL/Jour3


In [1]:
import duckdb
from skimpy import skim

In [2]:
# Création d'une BD virtuelle
conn = duckdb.connect()

## Fichier des ventes

In [3]:
# Requête SQL : récupération de toutes les données du fichier .csv en DF
sales_df = conn.execute(
    """
    SELECT *
    FROM read_csv_auto('data/indexE/Ventes.csv', header=True)
    """
).df()

In [4]:
# Stats de la DF ci-avant
skim(sales_df)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 100    │ │ int32       │ 5     │                                                          │
│ │ Number of columns │ 7      │ │ datetime64  │ 1     │                                                          │
│ └───────────────────┴────────┘ │ float64     │ 1     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column_name        ┃ NA  ┃ NA %   ┃ mean    ┃ sd     ┃ p0     ┃ p25    ┃ p50   ┃ p75    ┃ p100   ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━┩  │
│ │ VenteID            │   0 │      0 │    50.5 │  29.01 │      1 │  25.75 │  50.5 │  75.25 │    100 │ ▇▇▇▇▇▇  │  │
│ │ ClientID           │   0 │      0 │   51.26 │  29.18 │      1 │  27.75 │  53.5 │  77.25 │     99 │ ▆▇▅▇▆▇  │  │
│ │ EmployeID          │   0 │      0 │   50.65 │   29.4 │      2 │     23 │  49.5 │  76.25 │    100 │ ▇▇▇▆▇▇  │  │
│ │ ProduitID          │   0 │      0 │   48.08 │  24.56 │      3 │     29 │    46 │  65.25 │     98 │ ▅▆▇▇▅▃  │  │
│ │ QuantiteVendue     │   0 │      0 │   48.36 │  30.17 │      1 │     22 │  50.5 │     71 │    100 │ ▇▇▃▇▃▇  │  │
│ │ MontantTotal       │   0 │      0 │    8278 │  11580 │  59.98 │   1542 │  2979 │   9249 │  48500 │ ▇▁▁  ▁  │  │
│ └────────────────────┴─────┴────────┴─────────┴────────┴────────┴────────┴───────┴────────┴────────┴─────────┘  │
│                                                    datetime                                                     │
│ ┏━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓  │
│ ┃ column_name      ┃ NA   ┃ NA %    ┃ first                      ┃ last                       ┃ frequency    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩  │
│ │ DateVente        │    0 │       0 │    2020-01-08 12:43:33     │    2023-12-27 15:39:42     │ None         │  │
│ └──────────────────┴──────┴─────────┴────────────────────────────┴────────────────────────────┴──────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

In [5]:
# Enregistrement de la DF ci-avant en BD virtuelle (requête SQL : CTE)
conn.register('sales_db', sales_df)

## Requêtes SQL

Le n° ID des 10 employés qui réalisent le CAHT le plus élevé

In [7]:
conn.execute(
    """
    -- Champs à récupérer
    SELECT
        EmployeID
        , ROUND(SUM(MontantTotal), 0) AS "Total des ventes réalisées"
    FROM sales_db
    -- TCD
    GROUP BY EmployeID
    -- Trie
    ORDER BY "Total des ventes réalisées" DESC
    -- Affichage limité
    LIMIT 10
    """
).df()

,EmployeID,Total des ventes réalisées
0,97,93988.0
1,9,45999.0
2,36,45499.0
3,38,43999.0
4,67,34999.0
5,72,31309.0
6,12,28539.0
7,6,26159.0
8,100,25909.0
9,88,25899.0


Nombre de ventes réalisé par employé par ordre décroissant

In [8]:
conn.execute(
    """
    -- Champs à récupérer
    SELECT
        EmployeID
        , COUNT(DISTINCT(VenteID)) AS "Nombre de ventes réalisé"
    FROM sales_db
    -- TCD
    GROUP BY EmployeID
    -- Trie
    ORDER BY "Nombre de ventes réalisé" DESC
    """
).df()

,EmployeID,Nombre de ventes réalisé
0,48,3
1,97,3
2,33,3
3,23,3
4,72,3
...,...,...
60,14,1
61,62,1
62,30,1
63,75,1


Somme des ventes journalières par trie croissant

In [22]:
sales_pd = conn.execute(
    """
    -- Champs à récupérer
    SELECT
        DISTINCT(DateVente) AS Date
        , MontantTotal AS "Vente journalière"
    FROM sales_db
    -- Trie
    ORDER BY Date
    """
).df()

In [23]:
# Conversion du champ date en str
sales_pd['Date'] = sales_pd['Date'].dt.strftime('%d/%m/%Y')
sales_pd

,Date,Vente journalière
0,08/01/2020,2399.92
1,22/01/2020,389.87
2,02/02/2020,419.86
3,11/02/2020,30099.14
4,29/02/2020,2659.62
...,...,...
95,10/10/2023,2129.29
96,16/10/2023,2119.47
97,04/11/2023,6649.05
98,19/11/2023,2799.60


Somme des ventes par année par trie croissant

In [29]:
conn.execute(
    """
    -- Champs à récupérer
    SELECT
        YEAR(DateVente) AS Annee
        , ROUND(SUM(MontantTotal), 0) AS "CAHT annuel"
    FROM sales_db
    -- TCD
    GROUP BY Annee
    -- Trie
    ORDER BY "Annee" ASC
    """
).df()

,Annee,CAHT annuel
0,2020,210899.0
1,2021,219847.0
2,2022,279517.0
3,2023,117509.0


Moyenne des ventes par année et par employé

In [30]:
conn.execute(
    """
    -- Champs à récupérer
    SELECT
        EmployeID
        , YEAR(DateVente) AS Annee
        , ROUND(AVG(MontantTotal), 0) AS "Ventes moyennes"
    FROM sales_db
    -- TCD
    GROUP BY EmployeID, Annee
    -- Trie
    ORDER BY EmployeID ASC, Annee ASC
    """
).df()

,EmployeID,Annee,Ventes moyennes
0,2,2021,23799.0
1,3,2020,9740.0
2,6,2020,60.0
3,6,2021,6600.0
4,6,2022,19499.0
...,...,...,...
84,97,2020,990.0
85,97,2022,46499.0
86,99,2020,1949.0
87,99,2022,70.0


Donner la liste des 5 employés ayant réalisé le plus gros du CAHT

In [31]:
conn.execute(
    """
    -- Champs à récupérer
    SELECT
        EmployeID
        , ROUND(SUM(MontantTotal), 0) AS "Total des ventes réalisées"
    FROM sales_db
    -- TCD
    GROUP BY EmployeID
    -- Trie
    ORDER BY "Total des ventes réalisées" DESC
    -- Affichage limité
    LIMIT 5
    """
).df()

,EmployeID,Total des ventes réalisées
0,97,93988.0
1,9,45999.0
2,36,45499.0
3,38,43999.0
4,67,34999.0
